In [153]:
import pandas as pd
from sqlalchemy import create_engine  
from sqlalchemy import text


In [154]:
engine = create_engine("dsn") # replace this with db conn uri  
with engine.connect() as conn, conn.begin():  
    data = pd.read_sql_query("select id,job_title,job_link,locations,created_at,source from job_listings",conn)

In [155]:
data.head()

,id,job_title,job_link,locations,created_at,source
0,730146,Sales Engineering Manager,https://jobs.smartrecruiters.com/Zscaler/74399...,[Cleveland],2023-09-15 00:01:16.981078+00:00,trueup
1,1,Senior Software Engineer,https://boards.greenhouse.io/2k/jobs/5713141003,"[San Francisco, CA, USA]",2023-08-14 23:30:51.155051+00:00,simplify
2,105554,Senior Software Engineer - Mexico,https://boards.greenhouse.io/nubank/jobs/4884703,"[Mexico, Mexico City]",2023-08-17 05:16:33.505932+00:00,trueup
3,57465,Senior Security Engineer,https://boards.greenhouse.io/aurorainnovation/...,"[Mountain View, CA, USA]",2023-08-16 00:00:51.397802+00:00,simplify
4,6551,Senior Software Engineer,https://jobs.lever.co/localfoodgroup/5db55467-...,"[San Francisco, CA, USA]",2023-08-14 23:31:07.164960+00:00,simplify


In [156]:
import re
from urllib.parse import urlencode, urlparse, urlunparse, parse_qs

In [157]:
def strip_query_params(url, keys):
    u = urlparse(url)
    query = parse_qs(u.query, keep_blank_values=True)
    if type(keys) is list:
        for key in keys:
            query.pop(key, None)
        encoded_query = urlencode(query, True)
    elif keys==-1:    
        encoded_query = ""
    u = u._replace(query=encoded_query)
    return urlunparse(u)

In [158]:
def clean_url(url):
    pattern = re.compile("^https://jobs.lever.co/|https://boards.greenhouse.io/|https://jobs.ashbyhq.com/")
    if pattern.search(url) is not None:
        return strip_query_params(url,-1)
    return strip_query_params(url,["lever-origin", "lever-source", "lever-source[]", "source", "gh_src","utm_campaign","utm_content","utm_medium","utm_term","utm_source","iis","iisn"])
        

In [159]:
data['job_link'] = data['job_link'].apply(clean_url)

In [160]:
data['job_link'].count()

47733

In [161]:
len(data['job_link'].unique())

47733

In [162]:
data[data['job_link'].duplicated()].sort_values("id")


,id,job_title,job_link,locations,created_at,source


In [163]:
to_be_deleted = data[data['job_link'].duplicated()]['id'].astype(str).values.tolist()

In [164]:
sql_query = "DELETE FROM job_listings WHERE id in ({})".format(",".join(to_be_deleted))

In [165]:
sql_query

'DELETE FROM job_listings WHERE id in ()'

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text(sql_query))
    conn.commit()   

In [167]:
with engine.connect() as conn:
    sql_query = text('''UPDATE job_listings SET job_link=:job_link WHERE id=:id''')
    conn.execute(sql_query,data[['id','job_link']].to_dict('records'))
    conn.commit()

In [187]:
df_seen = pd.read_csv('./seen.csv',header=None,names=['job_link'])


In [188]:
df_seen.head()

,job_link
0,https://nvidia.wd5.myworkdayjobs.com/en-US/NVI...
1,https://nvidia.wd5.myworkdayjobs.com/en-US/NVI...
2,https://jobs.apple.com/en-us/details/200495908
3,https://recharge.recruitee.com/o/senior-produc...
4,https://www.mendel.ai/careers?gh_jid=4935548004


In [190]:
df_seen["job_link"] = df_seen["job_link"].apply(clean_url)

In [194]:
urls = df_seen["job_link"].unique()

In [198]:
df_seen.drop_duplicates(subset=['job_link'],inplace=True)

Populate job_id from job_links using the production database table

In [201]:
with engine.connect() as conn, conn.begin():  
    job_df = pd.read_sql_query("SELECT id,job_link from job_listings",conn)

In [203]:
df_user = pd.merge(df_seen, job_df, on='job_link', how='inner')
df_user

In [207]:
sql_query = text('''INSERT INTO user_jobs (user_id,job_id) VALUES (1,:id)''')
with engine.connect() as conn:
    conn.execute(sql_query,df_user[['id']].to_dict('records'))
    conn.commit()